In [ ]:
!pip install objectnat

In [ ]:
!pip install pyarrow 
#To read .parquet

In [1]:
import geopandas as gpd
gpd.read_file('examples_data/buildings.geojson').to_parquet('examples_data/buildings.parquet')
gpd.read_file('examples_data/services.geojson').to_parquet('examples_data/services.parquet')

In [2]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

buildings = gpd.read_parquet('examples_data/buildings.parquet')
services = gpd.read_parquet('examples_data/services.parquet')

polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001)  # Getting an "encircling" polygon around our layer to download the graph 
G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True)

buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True)

2024-09-12 17:37:40.391 | DEBUG    | iduedu.modules.drive_walk_builder:get_walk_graph:214 - Downloading walk graph from OSM ...
2024-09-12 17:37:40.392 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:70 - Started downloading and parsing walk graph...
2024-09-12 17:37:40.601 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:77 - Started downloading and parsing public trasport graph...


2024-09-12 17:37:47.314 | DEBUG    | iduedu.utils.utils:estimate_crs_for_bounds:35 - Estimated CRS for territory EPSG:32636


Parsing pt routes:   0%|          | 0/51 [00:00<?, ?it/s]

2024-09-12 17:37:49.895 | DEBUG    | iduedu.utils.utils:estimate_crs_for_bounds:35 - Estimated CRS for territory EPSG:32636


Calculating the weights of the walk graph:   0%|          | 0/46614 [00:00<?, ?it/s]

2024-09-12 17:37:54.940 | DEBUG    | iduedu.modules.drive_walk_builder:get_walk_graph:247 - Done!
2024-09-12 17:37:55.434 | DEBUG    | iduedu.modules.public_transport_builder:get_all_public_transport_graph:267 - Done!
2024-09-12 17:37:55.439 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:79 - Public trasport graph done!
2024-09-12 17:37:55.439 | DEBUG    | iduedu.modules.intermodal_builder:get_intermodal_graph:81 - Walk graph done!
2024-09-12 17:37:55.441 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:50 - Composing intermodal graph...
2024-09-12 17:37:56.966 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:179 - Done composing!


In [3]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gft_to=services,
                                                 to_nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 )

In [4]:
matrix.to_parquet('examples_data/matrix_time_big.parquet')